Created on Mon Dec 27 15:14:57 2021
@author: Anurag Srivastava

# Wildfire Ash Model

1. The model computes ash delivery from the hillslope using probabilistic approach with multi-year climate assuming fire happens every year on the same day.
2. The model works with a single OFE simulation of WEPP.
3. The model requires WEPP's daily water file.



Input: WEPP water file ('*_.wat.txt')

Output: Ash transport file ('*_ash.csv')


## Intuition

The WEPP model simulates runoff and sediment yield from different burn severities assuming uniform soil and vegetative characteristics. Following a wildfire, the presence of ash on the forest floor alters the runoff generation mechanism influencing the transport of ash along the hillslope. The interacting climate with eroding ash layer continually changes the physical characteristics of ash over time affecting its capacity to hold the water. The amount of runoff generated from the ash layer depends on ash characteristics (depth of ash, hydraulic conductivity,  bulk density, and particle density), rainfall timing and intensity, and their interactions. 

Following the fire, the presence of water repellent soils at the ash-soil layer interface prevents the percolation of water allowing more water to be stored within the ash layer because of a high porosity (low bulk density) that consequently reduces runoff through the ash layer. Although there is a reduction in runoff through the ash layer, ash particles are more highly erodible with intense rainfall or snowmelt events because of the presence of water-repellent soils underneath.   

As time progresses, successive rainfall or snowmelt events break the water repellent layers letting water percolate through the ash-soil interface, which results in the compaction of the ash layer reducing the porosity (increasing bulk density) of the ash layer. Precipitation events during this phase produce relatively higher runoff generated through the ash layer but become less erodible partly due to the lower amount of ash available for transport and consolidated ash layer.



## Assumptions

- Model operates on daily values of surface runoff and infiltration amount from WEPP.
- Infiltration is a function of rain+melt and surface runoff. Infiltrated water into the top soil layer is based on effective hydraulic conductivity of top mineral soil layer instead of effective hydraulic conductivity of ash layer.
- Bulk density of ash layer increases with time, which is a function of cumulative infiltration since the time of the fire day.
- Water holding capacity (porosity) of ash layer decreases over time with reduced ash depth.


## Model components

### Input Parameters

Following are the input parameters used for ash modeling. Parameter values currently are preliminary.

iniBulkDen = 0.18  # Initial bulk density ($\rho_i$), gm/cm3 

finBulkDen = 0.62  # Final bulk density ($\rho_f$), gm/cm3 

bulkDenFac = 0.005  # Bulk density factor ($\rho_{fact}$)

parDen = 1.2  # Ash particle density ($\rho_m$), gm/cm3 

decompFac = 0.00018  # Ash decomposition factor, t/ha/day

iniErod = 2  # Initial erodibility ($k_i$), t/ha

finErod = 0.01  # Final erodibility ($k_f$), t/ha

roughnessLimit = 1  # Threshold ash depth below which ash delivery is restricted ($R$), mm

iniAshDepth = 14  # Initial ash depth, mm

iniAshLoad = 10 * iniAshDepth * iniBulkDen  # Initial ash load, t/ha


### Infiltration

The ash model relies on modeled WEPP runoff from the burn severity soils and infiltrated amount for the computation of runoff amount from ash layer.

Daily infiltration (I) is computed as a difference of rain plus snowmelt ($RM$) and surface runoff ($Q$).  

$$I = RM - Q$$


### Bulk density


Bulk density is modeled using an exponential function to capture the consolidation of ash layer over time. Bulk density increases exponentially from the bulk density at time 0 (fire day) with increasing cumulative infiltration and asymptotically approaches the bulk density above a certain cumulative infiltration depending on the shape of the curve it follows.

Belk density is computed as:

$$\rho = \rho_f + (\rho_i - \rho_f) \; exp (- {\rho_{fact}}  * {I_{cum}})$$

where:

$\rho$ = bulk density of ash layer (gm/cm<sup>3</sup>)

$\rho_i$ = initial bulk density of ash layer on the day of fire (gm/cm<sup>3</sup>)

$\rho_f$ = asymptotic bulk density approached by the curve under a prolonged period of rainfall or infiltration (gm/cm<sup>3</sup>)

$\rho_{fact}$ = bulk density shape factor that defines the shape of the curve of $\rho$ versus $I_{cum}$ and ensure that bulk density approaches $\rho_f$ 

$I_{cum}$ = cumulative infiltration (mm)


### Porosity

Porosity of the ash layer is computed as a function of bulk density ($\rho$) and particle density. Particle density of the ash within the ash layer is assumed temporally uniform.

$$\Phi = 1 - \frac{\rho_b}{\rho_m}$$

where:

$\Phi$ = porosity of ash layer

$\rho_m$ = particle density of ash (gm/cm<sup>3</sup>)


### Ash runoff

The amount of runoff generated from the ash layer is computed by removing the depth of water that the ash layer can withhold from the runoff amount of WEPP as: 

$$Q_{ash, t-1} = Q_{t-1} - \Bigg(\frac{A_{ash, t-1}} {(10 * \rho_{t-1})}\Bigg) * \Phi$$

where:

$Q_{ash}$ = runoff amount from ash layer (mm)

$Q$ = runoff amount from WEPP (mm)

$A_{ash}$ = available ash amount (t/ha)

10 is conversion factor to convert ash amount in t/ha to ash depth in mm


### Ash delivery

Delivery of ash from the hillslope is calculated as runoff from the ash layer times transportable rate of ash. For higher values of normalized bulk density, ash delivery is governed by maximum ash transport rate while for lower values by asymptotic ash transport rate.  

$$DEL_{ash, t-1} = Q_{ash, t-1} * \Bigg[(k_i - k_f) * \Bigg(\frac{\rho_{t-1} - \rho_f} {\rho_i - \rho_f}\Bigg) + k_f \Bigg]$$

where:

$DEL_{ash}$ = ash delivery from hillslope (t/ha)

$k_i$ = maximum ash transport rate (t/ha)

$k_f$ = asymptotic ash transport rate (t/ha)

$\frac{\rho_{t-1} - \rho_f}{\rho_i - \rho_f}$ = normalized bulk density, ranges between 0-1


### Available ash

The available amount of ash on the current day is calculated by subtracting the ash delivered from the available ash amount on the previous day adjusted for the decomposition factor with infiltration. 


$$A_{ash, t} = A_{ash, t-1} * exp(-decompFac *  I_t) - DEL_{ash, t-1}, \; when \; DEP_{ash, t-1} > R$$
$$A_{ash, t} = 0, \; otherwise$$


where:

$decompFac$ = decay rate of available ash amount with infiltration (t/ha/day)

$DEP_{ash}$ = ash depth (mm)

$R$ = threshold value of ash depth below which ash transport is restricted with the assumption that the remaining ash mixes with the underneath mineral soil (mm) 


### Ash depth

Depth of ash layer for each time-step is computed as:

$$DEP_{ash, t} = \frac{A_{ash, t}} {10 * \rho_{t}} $$


## Model limitations

1) Particle density of ash generally increases as the ash particles become smaller over time. In this model, particle size of ash is assumed uniform. 

2) No evaporation from the ash layer is considered that could affect the runoff amount from the ash layer.

3) Runoff from the ash layer is computed by adjusting the WEPP modeleled runoff for burn severity with porosity. Sophisticated rainfall-infiltration-runoff models such as Green-ampt infiltration model should be considered for reliable estimation of runoff from ash layer.

4) No scheme of simulation of needle leaves over ash layer for moderate and low severity fire.



# Model Codes

In [3]:
# Import libraries
%matplotlib inline

import math
import os
import sys
import glob
import csv
# import dtale
import concurrent.futures
import seaborn as sns
import pandas as pd
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from time import time

pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
# Define fire day in julian
fireDay = 217
rotation_year = 2

**The fire day is '{{fireDay}}th' day of the year.**

In [5]:
# Define parameters
# iniBulkDen = 0.18  # Initial bulk density, gm/cm3
# finBulkDen = 0.62  # Final bulk density, gm/cm3
# bulkDenFac = 0.005  # Bulk density factor
# parDen = 1.2  # Ash particle density, gm/cm3
# decompFac = 0.00018  # Ash decomposition factor, per day
# iniErod = 1  # Initial erodibility, t/ha
# finErod = 0.01  # Final erodibility, t/ha
# roughnessLimit = 1  # Roughness limit, mm
# iniAshDepth = 14  # Initial ash depth, mm
# iniAshLoad = 10 * iniAshDepth * iniBulkDen  # Initial ash load, t/ha

In [6]:
def run_ash_model(wat):
    ''' Run ash model
    Parameters
    ----------

    Input: water file from WEPP'''   
    
    
    # Define parameters    
    if 'high' in dict_from_csv['1'][0]:
        iniBulkDen = 0.18  # Initial bulk density, gm/cm3
        finBulkDen = 0.62  # Final bulk density, gm/cm3
        bulkDenFac = 0.005  # Bulk density factor
        parDen = 1.2  # Ash particle density, gm/cm3
        decompFac = 0.00018  # Ash decomposition factor, per day
        iniErod = 2  # Initial erodibility, t/ha
        finErod = 0.01  # Final erodibility, t/ha
        roughnessLimit = 1  # Roughness limit, mm
        iniAshDepth = 30  # Initial ash depth, mm
        iniAshLoad = 10 * iniAshDepth * iniBulkDen  # Initial ash load, t/ha
    else:
        iniBulkDen = 0.30  # Initial bulk density, gm/cm3
        finBulkDen = 0.62  # Final bulk density, gm/cm3
        bulkDenFac = 0.005  # Bulk density factor
        parDen = 1.2  # Ash particle density, gm/cm3
        decompFac = 0.00018  # Ash decomposition factor, per day
        iniErod = 0.5  # Initial erodibility, t/ha
        finErod = 0.01  # Final erodibility, t/ha
        roughnessLimit = 1  # Roughness limit, mm
        iniAshDepth = 10  # Initial ash depth, mm
        iniAshLoad = 10 * iniAshDepth * iniBulkDen  # Initial ash load, t/ha
    
    
    # print(' ')
    # print('WEPP water file: ', os.path.basename(wat).split('.')[0])
    watr = pd.read_table(wat,
                         skiprows=range(0, 23),
                         sep='\s+',
                         header=None,
                         names=wat_col_names)


    # Make starting/ending date for stochastic climate
    if watr['Y_#'].iloc[0] == 1:
        starting = '1/1/' + str(watr['Y_#'].iloc[0] + 1900)
        ending = '12/31/' + str(watr['Y_#'].iloc[-1] + 1900)
    # Make starting/ending date for observed climate
    else:
        starting = '1/1/' + str(watr['Y_#'].iloc[0])
        ending = '12/31/' + str(watr['Y_#'].iloc[-1])

    # Create ash df
    ash_df = pd.DataFrame()

    # Get selected variables from watr df to ash df
    ash_df = watr[[
        'J_#', 'Y_#', 'P_mm', 'RM_mm', 'Q_mm', 'Total-Soil-Water_mm',
        'Snow-Water_mm'
    ]]

    # Insert date column to ash df
    ash_df.insert(0, 'Date', pd.date_range(start=starting, end=ending))

    # Define simulation length
    start_date = pd.to_datetime(starting)
    end_date = pd.to_datetime(ending)
    N = (end_date - start_date).days + 1
    idx_sim = (ash_df['Date'] >= start_date) & (ash_df['Date'] <= end_date)

    # Pre-compute some variables
    dates = ash_df.loc[idx_sim, 'Date'].values
    J = ash_df.loc[idx_sim, 'J_#'].values
    Y = ash_df.loc[idx_sim, 'Y_#'].values
    P = ash_df.loc[idx_sim, 'P_mm'].values
    RM = ash_df.loc[idx_sim, 'RM_mm'].values
    Q = ash_df.loc[idx_sim, 'Q_mm'].values
    TSW = ash_df.loc[idx_sim, 'Total-Soil-Water_mm'].values
    SWE = ash_df.loc[idx_sim, 'Snow-Water_mm'].values

    # Pre-allocate arrays with NaN values
    
    Year_rot = np.ones(N) * np.nan
    Julian_rot = np.ones(N) * np.nan    
    Orig_year = np.ones(N) * np.nan
    
    Julian = np.ones(N) * np.nan
    Year = np.ones(N) * np.nan
    Infil_mm = np.ones(N) * np.nan
    Cum_Infil_mm = np.ones(N) * np.nan
    Cum_Q_mm = np.ones(N) * np.nan
    Bulk_density_gmpcm3 = np.ones(N) * np.nan
    Porosity = np.ones(N) * np.nan
    Available_ash_tonspha = np.ones(N) * np.nan
    Ash_depth_mm = np.ones(N) * np.nan
    Ash_runoff_mm = np.ones(N) * np.nan
    Transport_tonspha = np.ones(N) * np.nan
    Water_transport_tonspha = np.ones(N) * np.nan
    Cum_ash_runoff_mm = np.ones(N) * np.nan
    Cum_water_transport_tonspha = np.ones(N) * np.nan    
    Wind_transport_tonspha = np.zeros(N)
    Cum_wind_transport_tonspha = np.zeros(N)
    Ash_transport_tonspha = np.ones(N) * np.nan
    Cum_ash_transport_tonspha = np.ones(N) * np.nan
    

    # Define initial conditions
    count = 0
    Year_rot[0] = 0
    Julian_rot[0] = 1    
    Orig_year[0] =  watr['Y_#'].iloc[0]
    
    Year[0] = watr['Y_#'].iloc[0] - 1
    Ash_depth_mm[0] = iniAshDepth
    Available_ash_tonspha[0] = iniAshLoad
    Julian[0] = 1
    Infil_mm[0] = 0
    Cum_Infil_mm[0] = 0
    Cum_Q_mm[0] = 0
    Bulk_density_gmpcm3[0] = iniBulkDen
    Porosity[0] = 1 - (Bulk_density_gmpcm3[0] / parDen)
    Cum_ash_runoff_mm[0] = 0
    Cum_water_transport_tonspha[0] = 0
    

    # Model
    for t in range(1, N):
                
        Orig_year[t] = Y[t]

        if J[t] >= fireDay:
            Year[t] = Y[t]
        else:
            Year[t] = Y[t] - 1

        if J[t] == fireDay: 
            Julian[t] = 1 
        else:
            Julian[t] = Julian[t - 1] + 1            

        if J[t] == fireDay:
            count = count + 1
            if (count == 1) or (count > rotation_year):
                count = 1            
                Julian_rot[t] = 1
            else:
                Julian_rot[t] = Julian_rot[t - 1] + 1
        else:
            Julian_rot[t] = Julian_rot[t - 1] + 1            
        Year_rot[t] = count

        
        Infil_mm[t] = RM[t] - Q[t]

        if Julian_rot[t] == 1:
            Cum_Infil_mm[t] = Infil_mm[t]
            Cum_Q_mm[t] = Q[t]
            Bulk_density_gmpcm3[t] = iniBulkDen
            Porosity[t] = 1 - (Bulk_density_gmpcm3[t] / parDen)
        else:
            Cum_Infil_mm[t] = Cum_Infil_mm[t - 1] + Infil_mm[t]
            Cum_Q_mm[t] = Cum_Q_mm[t - 1] + Q[t]
            Bulk_density_gmpcm3[t] = finBulkDen + (
                iniBulkDen - finBulkDen) * np.exp(
                    -bulkDenFac * Cum_Infil_mm[t])
            Porosity[t] = 1 - (Bulk_density_gmpcm3[t] / parDen)

        if Q[t - 1] > (Available_ash_tonspha[t - 1] /
                       (10. * Bulk_density_gmpcm3[t - 1])) * Porosity[t - 1]:
            Ash_runoff_mm[t - 1] = np.maximum(
                0, Q[t - 1] -
                (Available_ash_tonspha[t - 1] /
                 (10. * Bulk_density_gmpcm3[t - 1])) * Porosity[t - 1])
        else:
            Ash_runoff_mm[t - 1] = 0

        if Ash_runoff_mm[t - 1] > 0:
            Transport_tonspha[t - 1] = (iniErod - finErod) * (
                Bulk_density_gmpcm3[t - 1] -
                finBulkDen) / (iniBulkDen - finBulkDen) + finErod
            
            Water_transport_tonspha[t - 1] = np.maximum(
                0,
                np.minimum(Available_ash_tonspha[t - 1], Ash_runoff_mm[t - 1] *
                           Transport_tonspha[t - 1]))
            
            Wind_transport_tonspha[t - 1] = 0
                        
        else:
            Transport_tonspha[t - 1] = 0
            Water_transport_tonspha[t - 1] = 0
            
            Wind_transport_tonspha[t - 1] = 0
            Ash_transport_tonspha[t - 1] = 0
            
        Ash_transport_tonspha[t - 1] = Wind_transport_tonspha[t - 1] + Water_transport_tonspha[t - 1]

        if Julian_rot[t] > 1:

            if Ash_depth_mm[t - 1] < roughnessLimit:
                Available_ash_tonspha[t] = 0
            else:
                Available_ash_tonspha[t] = Available_ash_tonspha[t - 1] * np.exp(-decompFac * Infil_mm[t]) - \
                    Water_transport_tonspha[t - 1]
        else:
            Available_ash_tonspha[t] = iniAshLoad

        Ash_depth_mm[t] = Available_ash_tonspha[t] / (10. *
                                                      Bulk_density_gmpcm3[t])

        if Julian_rot[t] == 1:
            Cum_ash_runoff_mm[t] = Ash_runoff_mm[t - 1]
            Cum_wind_transport_tonspha[t] = 0
            Cum_water_transport_tonspha[t] = Water_transport_tonspha[t - 1]
        else:
            Cum_ash_runoff_mm[t] = Cum_ash_runoff_mm[t - 1] + \
                Ash_runoff_mm[t - 1]
            Cum_wind_transport_tonspha[t] = 0
            Cum_water_transport_tonspha[t] = Cum_water_transport_tonspha[
                t - 1] + Water_transport_tonspha[t - 1]
            
        Cum_ash_transport_tonspha[t] = Cum_wind_transport_tonspha[t] + Cum_water_transport_tonspha[t]
        
        

    # convert numpy arrays to pandas dataframe
    my_array = np.array([
        Orig_year, Year, Year_rot, Julian_rot, Julian, P, RM, SWE, Q, TSW, Infil_mm, Cum_Infil_mm, Cum_Q_mm,
        Bulk_density_gmpcm3, Porosity, Available_ash_tonspha, Ash_depth_mm,
        Ash_runoff_mm, Transport_tonspha, Cum_ash_runoff_mm, Water_transport_tonspha,
        Wind_transport_tonspha, Ash_transport_tonspha,
        Cum_water_transport_tonspha, Cum_wind_transport_tonspha, Cum_ash_transport_tonspha  
    ]).T

    index = list(range(0, len(Year)))

    columns = [
        'orig_year', 'year', 'Year_rot', 'julian_rot', 'julian', 'precip (mm)', 'rainmelt (mm)', 'snow water equivalent (mm)', 'runoff (mm)', 'tot soil water (mm)',
        'infiltration (mm)', 'cum_infiltration (mm)', 'cum_runoff (mm)', 'bulk density (gm/cm3)',
        'porosity', 'available ash (tonne/ha)', 'ash depth (mm)', 'ash runoff (mm)',
        'transport (tonne/ha)', 'cum_ash_runoff (mm)',
        'water_transport (tonne/ha)', 'wind_transport (tonne/ha)', 'ash_transport (tonne/ha)', 
        'cum_water_transport (tonne/ha)', 'cum_wind_transport (tonne/ha)', 'cum_ash_transport (tonne/ha)'        
    ]

    df = pd.DataFrame(my_array, index, columns)

    # remove the first and the last year
    df = df[(df['year'] >= df['year'].iloc[0])
            & (df['year'] <= df['year'].iloc[-1])]

    # reset and rename index
    df.reset_index(drop=True, inplace=True)
    df.index.rename("sno", inplace=True)    
    
    # determine site number
    siteno = os.path.basename(wat).split('.')[0][:-4]
    
    # determine site name
    sitename = dict_from_csv[str(os.path.basename(wat).split('.')[0][:-4])][7].strip()
    
    # determine burn severity
    severity = dict_from_csv[str(os.path.basename(wat).split('.')[0][:-4])][0].split('.')[0]

    # Update date
    df.insert(0, 'date', pd.date_range("01-01-" + str(int(df['year'].iloc[0])), periods=len(df), freq='D'))
    
    # Update original date
    df.insert(0, 'orig_date', pd.date_range("01-01-" + str(int(df['orig_year'].iloc[0])), periods=len(df), freq='D'))

    # write ash output file
    df.to_csv(os.path.join(out_path,
        siteno + '_' + sitename + '_' + severity + '_ash.csv'))


In [7]:
if __name__ == '__main__':

    ''' Reads WEPP water files from ./out/ folder'''

    ######## Change working dir ##########################
    wdir = r'F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope'
    os.chdir(wdir)
    # print(wdir)
    ######################################################

    out_path = os.path.join(wdir, 'out')

    # process watr.txt file to get SWE and compare it with observed SWE
    print()
    # Header row
    wat_col_names = "OFE J Y P RM Q Ep Es Er Dp UpStrmQ SubRIn latqcc Total-Soil-Water frozwt Snow-Water QOFE Tile Irr Area"
    wat_col_names = wat_col_names.split()

    # Units row
    wat_units = "# # # mm mm mm mm mm mm mm mm mm mm mm mm mm mm mm mm m^2"
    wat_units = wat_units.split()

    # Concatenating header and units
    # def concat_func(x, y):
    #     return x + " (" + y + ")"
    def concat_func(x, y): return x + "_" + y
    wat_col_names = list(map(concat_func, wat_col_names, wat_units))

    
    # get run names for graphical view 
    with open('run_input_file.csv', mode='r') as inp:
        reader = csv.reader(inp)
        next(reader)
        dict_from_csv = {rows[0]:rows[1:] for rows in reader}
#     print(dict_from_csv)

    # Sequential runs
    for wat in glob.glob(os.path.join(wdir, "out", "*_wat.txt")):
        print(wat)
        run_ash_model(wat)


F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\10_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\11_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\12_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\13_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\14_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\1_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\2_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\continuous_hillslope\out\3_wat.txt
F:\PROJECTS\WEPP_CLOUD\WEPPcloud_Riverside_fire\ASH_MODEL_WEPP\Jonay\9_22_2021\con